# Introduction

## Présentation

Redis est une base de données NoSQL qui se classe dans la catégorie "data structure server". C'est  un système permettant de traiter, de manière très performante, des données sous la forme clés/valeurs. Cette performance est obtenue car les données sont stockées directement sur la mémoire RAM. Cela permet d'accéder très facilement et très rapidement aux données 

La clé permet de récupérer et d'identifier de manière unique la donnée. Redis ne permet pas de stocker des modèles de données très complexes, il faut donc ruser pour créer une structure de données complète et complexe.

Redis est souvent utilisé dans le developpement web pour gérer les sessions utilisateurs, leur panier, leurs configurations. En outre, lorsqu'on stocke une petite quantité de donnée et qu'on a besoin d'y accèder en quelques milliseconds

## Les données 



Les valeurs peuvent appartenir aux types suivants :

- chaînes de caractères
- compteurs numériques (les données numériques seront stockées dans la base sous la forme de chaîne de caractères)
- listes 
- sets ou sets ordonnés
- dictionnaires

## Lancer un container Redis

Comme pour Mongo, il nous faut une instance redis qui tourne en local, pour se faire il suffit de lancer un container Redis

Vous devriez être familier avec Docker maintenant, à vous de jouer pour lancer ce container !

## Installation

Le package Redis pour python est déjà installé, voici la commande si jamais :

`pipenv install redis==5.0.8`

# Let's play

In [2]:
import redis

In [3]:
redis_client = redis.StrictRedis(host='localhost')

Pour tester la connexion on peut pinguer la base de données :

In [4]:
redis_client.ping()

True

Vous pouvez regarder toutes les configurations utilisées par Redis, beaucoup ne sont pas utiles à notre niveau, mais vous pouvez voir par exemple, `database`, `maxclients`, `port` , `timeout` qui peuvent être utile de modifier pour différentes applications

In [5]:
redis_client.config_get()

{'appendonly': 'no',
 'aof-timestamp-enabled': 'no',
 'search-raw-docid-encoding': 'no',
 'busy-reply-threshold': '5000',
 'tls-client-cert-file': '',
 'appendfilename': 'appendonly.aof',
 'aof-load-corrupt-tail-max-size': '0',
 'replica-announce-port': '0',
 'cluster-link-sendbuf-limit': '0',
 'tracking-table-max-keys': '1000000',
 'maxmemory-policy': 'noeviction',
 'search-min-stem-len': '4',
 'cluster-announce-human-nodename': '',
 'search-default-scorer': 'BM25STD',
 'tls-protocols': '',
 'slave-lazy-flush': 'no',
 'rdbcompression': 'yes',
 'lazyfree-lazy-user-del': 'no',
 'auto-aof-rewrite-min-size': '67108864',
 'proto-max-bulk-len': '536870912',
 'protected-mode': 'no',
 'aof-use-rdb-preamble': 'yes',
 'aclfile': '',
 'ts-encoding': 'compressed',
 'latency-tracking-info-percentiles': '50 99 99.9',
 'databases': '16',
 'search-_prioritize-intersect-union-children': 'no',
 'cluster-require-full-coverage': 'yes',
 'maxmemory-eviction-tenacity': '10',
 'latency-monitor-threshold': '

## Stockage clé valeurs

Pour stocker une valeur correspondant à une clé 

In [6]:
KEY = "name"
VALUE = "yourname"
redis_client.set(KEY, VALUE)

True

True montre que le stockage s'est bien déroulé. 

Pour récupérer la valeur stockée il suffit d'appeler la méthode `GET` sur cette clé

In [7]:
redis_client.get(KEY)

b'yourname'

Vous pouvez voir qu'à l'aide du `b` que la réponse n'est pas exactement ce que l'on a stocké, mais une représentation en bytes de notre chaine de charactères. 

In [8]:
assert redis_client.get(KEY) == VALUE, "Les valeurs ne correspondent pas"

AssertionError: Les valeurs ne correspondent pas

Pour cela, en python, il faut appeler la méthode `decode()` pour transformer des bytes en string 

In [ ]:
assert redis_client.get(KEY).decode() == VALUE, "les valeurs ne correspondent pas"

On peut préciser au client de décoder les réponses et de les transformer directement en strings

In [ ]:
redis_client = redis.StrictRedis(host='localhost', charset='utf-8', decode_responses=True)

In [ ]:
assert redis_client.get(KEY) == VALUE

Les valeurs peuvent aussi être des entiers

In [ ]:
KEY = "age"
VALUE = 20

redis_client.set(KEY, VALUE)

True

In [ ]:
redis_client.get(KEY)

'20'

Vous pouvez alors utiliser la méthode `incr` pour incrémenter cet entier. Il faut faire attention puisque redis renvoie toujours les entiers comme des chaines de charactères.

In [ ]:
redis_client.incr(KEY)

22

## Exemple 

In [11]:
from ipywidgets import widgets, interact
from IPython.display import display

In [12]:
KEY = "click_number"
DEFAULT_VALUE = 0

redis_client.set(KEY, DEFAULT_VALUE)

button = widgets.Button(description="Click me!")
def on_button_clicked(button):
    redis_client.incr(KEY)
    print(f"Le bouton à été cliqué {redis_client.get(KEY)} fois")
    button.description = "Click me more !"
    
button.on_click(on_button_clicked)
display(button)

Button(description='Click me!', style=ButtonStyle())

In [13]:
print(f"Tu as cliqué sur le bouton {redis_client.get(KEY)} fois !")

Tu as cliqué sur le bouton b'32' fois !


## Les Listes

Redis permet aussi de stocker des valeurs de type `list`, pour cela nous avons à notre disposition un ensemble de méthodes :  

- La méthode `LPUSH`  permet d'ajouter un élément en tête de list la complexité est de O(1).  
- La méthode `RPUSH` permet d'ajouter un élément en queue de list la complexité est de O(1).  
- La méthode `LINSERT` permet d'ajouter un élément avant ou après la première occurence d'une certaine valeur, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `LSET` permet de modifier la valeur correspondant à l'index spécifié, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `RPOP` permet de supprimer un élément en queue de list la complexité est de O(1).
- La méthode `LPOP` permet de supprimer un élément en tête de list la complexité est de O(1).
- La méthode `LINDEX` permet d'afficher un élément en fonction de son index.
- La méthode `LREM` permet de supprimer un élément en fonction de son index.
- La méthode `LLEN` permet de récupérer la taille d'une liste.

Nous allons toutes les tester.

In [14]:
redis_client.lpush('language_list', "kotlin")
redis_client.lpush('language_list', "python")

2

On peut aussi ajouter plusieurs éléments en même temps dans une liste

In [15]:
redis_client.lpush('language_list', "java", "c++")

4

In [16]:
f"La taille de la liste est {redis_client.llen('language_list')}"

'La taille de la liste est 4'

In [17]:
INDEX = 2
f"L'élément à l'index {INDEX} de la liste est '{redis_client.lindex('language_list', INDEX)}'"

"L'élément à l'index 2 de la liste est 'b'python''"

Le client renvoie la taille de la liste venant d'être mise à jour. On peut afficher tous les éléments de la liste en les retirant un par un.

In [18]:
while(redis_client.llen('language_list')!=0):

    print(redis_client.lpop('language_list'))

b'c++'
b'java'
b'python'
b'kotlin'


## Les dictionnaires 

Redis permet de stocker assez facilement des dictionnaires, on peut comme cela stocker des structures de données plus complexes. Pour cela, nous avons plusieurs méthodes : 

- La méthode `HMSET` permet d'ajouter un dictionnaire à une clé
- La méthode `HSET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGETALL` permet de récupérer toutes les clés valeurs d'un dictionnaire.
- La méthode `HKEYS` permet de récupérer toutes les clés d'un dictionnaire
- La méthode `HVALS` permet de récupérer toutes les valeurs d'un dictionnaire

In [19]:
doc = {
    "name":"yourname",
    "age":20,
    "city":"paris"
}

In [20]:
redis_client.hset("user1", mapping=doc)

3

In [21]:
redis_client.hgetall("user1")

{b'name': b'yourname', b'age': b'20', b'city': b'paris'}

On peut modifier uniquement une clé du dictionnaire

In [22]:
redis_client.hset("user1", "city", "lyon")

0

In [23]:
redis_client.hgetall("user1")

{b'name': b'yourname', b'age': b'20', b'city': b'lyon'}

## Les pipelines

In [24]:
import timeit

Lorsque l'on appelle une méthode depuis le client python, chaqu'une d'entre elles fait un appel à Redis, ce qui peut prendre du temps. Les pipelines permettent de réduire ce temps en stockant toutes les opérations et en les applicant une seule fois sur la base. Cela permet d'augmenter grandement les performances en réduisant les appels et le passage par le réseau. 

In [25]:
N = 10000

In [26]:
def store_key_values(N):
    for i in range(N):
        redis_client.set(f"key_{i}", f"value_{i}")

In [27]:
%%timeit
store_key_values(N=N)

4.21 s ± 547 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
def store_key_values_with_pipelines(N):
    p = redis_client.pipeline()
    for i in range(N):
        p.set(f"key_{i}", f"value_{i}")

In [29]:
%%timeit
store_key_values_with_pipelines(N=N)

KeyboardInterrupt: 

On voit alors que l'on peut diviser le temps par 100. Dans vos différents métiers les performances sont souvent très importantes

# Exercice

Utiliser redis pour gérer les commandes d'un restaurant de burgers, pour simplifier les choses les clients pourront choisir leurs ingrédients dans 3 types de produits :

- Pain: blanc, complet, aux céréales
- Garniture : salade, tomates , oignons, concombres
- Protéines : poulet, boeuf, agneau, tofu

Chaque ingrédient équivaut à un prix et un poids, avec chaque commande, vous devrez calculer le poids et le prix que les clients devront régler. 

1) Stocker les informations concernant les ingrédients dans redis   
2) Créer la méthode permettant de créer aléatoirement un burger  
3) Créer la méthode qui permet d'empiler dans redis les prochaines commandes (Réfléchir au type de données utilisé)  
4) Créer la méthode permettant de calculer le poids et le prix du burger  
5) Créer une boucle pour dépiler, réaliser et servir les burgers.  

Pour vous aider voici quelques données 

In [30]:
pains = {
    "blanc":{
        "poid":100,
        "prix": 2
    },
    "complet":{
        "poid":90,
        "prix": 3
    },
    "cereales":{
        "poid":120,
        "prix": 4
    }
}

In [31]:
garniture = {
    "salade":{
        "poid":50,
        "prix": 1
    }, "tomates":{
        "poid":60,
        "prix": 2
    } , "oignons":{
        "poid":10,
        "prix": 1.5
    }, "concombres":{
        "poid":30,
        "prix": 3
    }
}

In [32]:
proteine = {
    "poulet":{
        "poid":100,
        "prix": 6
    }, "boeuf":{
        "poid":150,
        "prix": 5
    } , "agneau":{
        "poid":120,
        "prix": 7
    }, "tofu":{
        "poid":80,
        "prix": 8
    }
}

In [46]:
import random
import json

def make_burger():
    """Crée un burger aléatoire avec 1 pain, 1-3 garnitures, et 1 protéine"""
    pain_choice = random.choice(list(pains.keys()))
    garniture_choices = random.sample(list(garniture.keys()), random.randint(1, 3))
    proteine_choice = random.choice(list(proteine.keys()))
    
    burger = {
        "pain": pain_choice,
        "garnitures": garniture_choices,
        "proteine": proteine_choice
    }
    return burger

def make_order():
    """Crée un burger et l'ajoute à la file d'attente des commandes"""
    burger = make_burger()
    burger_json = json.dumps(burger)
    redis_client.rpush("orders_queue", burger_json)

def get_next_order():
    """Dépile et retourne la prochaine commande de la file d'attente"""
    order_json = redis_client.lpop("orders_queue")
    if order_json:
        return json.loads(order_json)
    return None

def process_price_weight(burger):
    """Calcule le prix total et le poids total d'un burger"""
    total_price = 0
    total_weight = 0
    
    # Ajouter le pain
    pain_data = redis_client.hgetall(f"pain:{burger['pain']}")
    if not pain_data:
        raise ValueError(f"Pain '{burger['pain']}' non trouvé dans Redis. Exécutez d'abord la cellule de stockage des ingrédients.")
    total_price += float(pain_data['prix'])
    total_weight += float(pain_data['poid'])
    
    # Ajouter les garnitures
    for garniture_name in burger['garnitures']:
        garniture_data = redis_client.hgetall(f"garniture:{garniture_name}")
        if not garniture_data:
            raise ValueError(f"Garniture '{garniture_name}' non trouvée dans Redis.")
        total_price += float(garniture_data['prix'])
        total_weight += float(garniture_data['poid'])
    
    # Ajouter la protéine
    proteine_data = redis_client.hgetall(f"proteine:{burger['proteine']}")
    if not proteine_data:
        raise ValueError(f"Protéine '{burger['proteine']}' non trouvée dans Redis.")
    total_price += float(proteine_data['prix'])
    total_weight += float(proteine_data['poid'])
    
    return total_price, total_weight

def process_burger():
    """Dépile une commande, calcule son prix/poids et retourne les infos"""
    burger = get_next_order()
    if burger is None:
        return None, None, None
    
    price, weight = process_price_weight(burger)
    burger_description = f"Pain {burger['pain']}, {', '.join(burger['garnitures'])}, {burger['proteine']}"
    
    return burger_description, price, weight

In [51]:
# Recréer le client Redis avec decode_responses=True
redis_client = redis.StrictRedis(host='localhost', charset='utf-8', decode_responses=True)

# 1) Stocker les informations des ingrédients dans Redis
for pain_name, pain_data in pains.items():
    redis_client.hset(f"pain:{pain_name}", mapping=pain_data)

for garniture_name, garniture_data in garniture.items():
    redis_client.hset(f"garniture:{garniture_name}", mapping=garniture_data)

for proteine_name, proteine_data in proteine.items():
    redis_client.hset(f"proteine:{proteine_name}", mapping=proteine_data)

print("✅ Ingrédients stockés dans Redis")

# Créer quelques commandes pour tester
for i in range(5):
    make_order()
print("✅ 5 commandes créées dans la file d'attente")

✅ Ingrédients stockés dans Redis
✅ 5 commandes créées dans la file d'attente


In [52]:
# 5) Boucle pour dépiler, réaliser et servir les burgers
while True:
    burger, price, weight = process_burger()
    if burger is None:
        print("✅ Toutes les commandes ont été servies!")
        break
    print(f'Le burger {burger} de {weight} gr a été préparé et servi, il faut régler {price}€')

Le burger Pain complet, tomates, boeuf de 300.0 gr a été préparé et servi, il faut régler 10.0€
Le burger Pain cereales, oignons, boeuf de 280.0 gr a été préparé et servi, il faut régler 10.5€
Le burger Pain complet, oignons, tomates, concombres, agneau de 310.0 gr a été préparé et servi, il faut régler 16.5€
Le burger Pain blanc, salade, tomates, oignons, boeuf de 370.0 gr a été préparé et servi, il faut régler 11.5€
Le burger Pain blanc, oignons, concombres, agneau de 260.0 gr a été préparé et servi, il faut régler 13.5€
Le burger Pain complet, oignons, tomates, salade, tofu de 290.0 gr a été préparé et servi, il faut régler 15.5€
Le burger Pain complet, salade, tomates, oignons, agneau de 330.0 gr a été préparé et servi, il faut régler 14.5€
Le burger Pain blanc, salade, tomates, poulet de 310.0 gr a été préparé et servi, il faut régler 11.0€
Le burger Pain cereales, salade, concombres, oignons, agneau de 330.0 gr a été préparé et servi, il faut régler 16.5€
Le burger Pain cereales,